In [1]:
from alpaca.data.live import CryptoDataStream, StockDataStream
import Authorization
import asyncio

In [2]:
# Asign Key and secret value
API_KEY = Authorization.ALPACA_API_KEY
SECRET_KEY = Authorization.ALPACA_SECRET_KEY
# Initiate class
crypto_stream = CryptoDataStream(API_KEY, SECRET_KEY)
Stock_stream = StockDataStream(API_KEY, SECRET_KEY)

In [ ]:
# define callback function to print the bar upon reciving.
async def bar_callback(bar):
    for property_name, value in bar:
        print(f"\"{property_name}\": {value}")
        
# Subscribing to bar event 
symbol = "BTC/USD"
crypto_stream.subscribe_bars(bar_callback, symbol)
crypto_stream.run()

In [4]:
from alpaca.data.live import StockDataStream

In [5]:
stock_stream = StockDataStream(API_KEY, SECRET_KEY)

In [ ]:
# async handler
async def quote_data_handler(data: any):
    # quote data will arrive here
    print(data)
    
symbols = 'TSLA'
stock_stream.subscribe_quotes(quote_data_handler,symbols)

stock_stream.run()

In [ ]:
# Stock RealTime Data using websocket.
from websocket import create_connection
import simplejson as json
import pprint

url = 'wss://stream.data.alpaca.markets/v2/iex'
ws = create_connection(url)
print(json.loads(ws.recv())) #print the connection message

auth_message = {"action": "auth", "key": API_KEY, "secret": SECRET_KEY}
ws.send(json.dumps(auth_message))
print(json.loads(ws.recv())) # print the authenticated message

subscription = {"action":"subscribe","trades":["AAPL"],"quotes":["AMD","CLDR"],"bars":["AAPL","VOO"]}
ws.send(json.dumps(subscription))
print(json.loads(ws.recv())) # print the subscribed message

while True:
    data = json.loads(ws.recv())
    pprint.pprint(data[0])
    print('*********************')

In [ ]:
# Crypto RealTime Data using websocket.
import Authorization
from ConnectDB import *
from websocket import create_connection
import simplejson as json
import pprint

# Asign Key and secret value
API_KEY = Authorization.ALPACA_API_KEY
SECRET_KEY = Authorization.ALPACA_SECRET_KEY

# Connect DataBase
con,cur = connect_data_base()
# prepare table_name as today's date
quote_table_name = "crypto_quote_" + datetime.date.today().strftime('%Y_%m_%d')
trade_table_name = "crypto_trade_" + datetime.date.today().strftime('%Y_%m_%d')
bar_table_name = "crypto_bar_" + datetime.date.today().strftime('%Y_%m_%d')
# Create table if not esists,
cur.execute(
    "CREATE TABLE IF NOT EXISTS %s (symbol TEXT, time TIMESTAMPTZ, type TEXT, ask NUMERIC,\
    ask_size NUMERIC, bid NUMERIC, bid_size NUMERIC);" %quote_table_name)
cur.execute(
    "CREATE TABLE IF NOT EXISTS %s (symbol TEXT, time TIMESTAMPTZ, type TEXT, id NUMERIC,\
    price NUMERIC, size NUMERIC, side TEXT);" %trade_table_name)
cur.execute(
    "CREATE TABLE IF NOT EXISTS %s (symbol TEXT, time TIMESTAMPTZ, type TEXT, close NUMERIC,\
     high NUMERIC, low NUMERIC,open NUMERIC, trade NUMERIC, volume NUMERIC, vwap NUMERIC);" %bar_table_name)
# excute SQL command
con.commit()

url = 'wss://stream.data.alpaca.markets/v1beta2/crypto'
ws = create_connection(url)
print(json.loads(ws.recv())) #print the connection message

auth_message = {"action": "auth", "key": API_KEY, "secret": SECRET_KEY}
ws.send(json.dumps(auth_message))
print(json.loads(ws.recv())) # print the authenticated message

subscription = {"action":"subscribe","trades":["BTC/USD"],"quotes":["BTC/USD"],"bars":["AM.BCH/USD"]}
ws.send(json.dumps(subscription))
print(json.loads(ws.recv())) # print the subscribed message

while True:
    # Receive and print data.
    data = json.loads(ws.recv())
    pprint.pprint(data[0])
    print('*********************')
    
    # Data organized.
    if data[0]['T'] == 'q':        
        Symbol = data[0]['S']
        Type = data[0]['T']
        Ask = data[0]['ap']
        Ask_size = data[0]['as']
        Bid = data[0]['bp']
        Bid_size = data[0]['bs']
        Time = data[0]['t']
        cur.execute("INSERT INTO {} (symbol, Time, Type, Ask, Ask_size, Bid, Bid_size) VALUES (%s, %s, %s, %s, %s, %s, %s)".format(quote_table_name),
        (Symbol, Time, Type, Ask, Ask_size, Bid, Bid_size))
        # excute query
        con.commit()
    
    elif data[0]['T'] == 't':
        Symbol = data[0]['S']
        Type = data[0]['T']
        Trade_id = data[0]['i']
        Trade_price = data[0]['p']
        Trade_size = data[0]['s']
        Taker_side  = data[0]['tks']
        Time = data[0]['t']
        # Insert all data to database and excute.  
        cur.execute("INSERT INTO {} (symbol, time, type, id, price, size, side) VALUES (%s, %s, %s, %s, %s, %s, %s)".format(trade_table_name),
        (Symbol, Time, Type, Trade_id, Trade_price, Trade_size, Taker_side))
        con.commit()
        
    elif data[0]['T'] == 'b':
        Symbol = data[0]['S']
        Type = data[0]['T']
        Close_price = data[0]['c']
        High_price = data[0]['h']
        Low_price = data[0]['l']
        Trade_count  = data[0]['n']
        Open_price = data[0]['o']
        Time = data[0]['t']
        Volume = data[0]['v']
        Vwap = data[0]['vw']
        # Insert all data to database and excute.  
        cur.execute("INSERT INTO {} (symbol, time, type, close, high, low, open, trade, volume, vwap) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(bar_table_name),
        (Symbol, Time, Type, Close_price, High_price, Low_price, Open_price, Trade_count, Volume, Vwap))
        con.commit()        
        


In [8]:
import pandas as pd
import datetime
Time = '2022-11-12T01:17:10Z'

In [9]:
NewYork_Time = pd.Timestamp(Time, tz="America/New_York").isoformat()

In [13]:
NewYork_Time.split('T')[0].replace('-', '_')

'2022_11_11'